In [ ]:
import hail as hl
from pcgc_hail.hail_scripts.utils import *
from pcgc_hail.hail_scripts.annotate_frequencies import *

# The master will need to be edited on each run
hl.init(master='spark://c18n05.ruddle.hpc.yale.internal:7077')

# Starting with the VEP annotated matrix table
mt = hl.read_matrix_table('/gpfs/ycga/scratch60/ysm/lek/njl27/gruber_exomes/spark_on_slurm/hail_output/gruber_WES_b5_vep.mt')

# sort transcript consequences 
mt = mt.annotate_rows(
    sortedTranscriptConsequences=get_expr_for_vep_sorted_transcript_consequences_array(vep_root=mt.vep)
)

# annotate rows with index zero element of array sorted transcript annotations
# hl.null was replaced with hl.missing as the former has been deprecated
# The code below is ugly and the checks are redundant and possibly overkill
mt = mt.annotate_rows(
    gene_symbol=hl.if_else(mt.sortedTranscriptConsequences.size() > 0, mt.sortedTranscriptConsequences[0].gene_symbol, hl.missing(hl.tstr)), 
    ENSG=hl.if_else(mt.sortedTranscriptConsequences.size() > 0, mt.sortedTranscriptConsequences[0].gene_id, hl.missing(hl.tstr)), 
    ENST=hl.if_else(mt.sortedTranscriptConsequences.size() > 0, mt.sortedTranscriptConsequences[0].transcript_id, hl.missing(hl.tstr)), 
    major_consequence=hl.if_else(mt.sortedTranscriptConsequences.size() > 0, mt.sortedTranscriptConsequences[0].major_consequence, hl.missing(hl.tstr)), 
    hgvsc=hl.if_else(mt.sortedTranscriptConsequences.size() > 0, mt.sortedTranscriptConsequences[0].hgvsc, hl.missing(hl.tstr)), 
    hgvs=hl.if_else(mt.sortedTranscriptConsequences.size() > 0, mt.sortedTranscriptConsequences[0].hgvs, hl.missing(hl.tstr)), 
    category=hl.if_else(mt.sortedTranscriptConsequences.size() > 0, mt.sortedTranscriptConsequences[0].category, hl.missing(hl.tstr)), 
    canonical=hl.if_else(mt.sortedTranscriptConsequences.size() > 0, mt.sortedTranscriptConsequences[0].canonical, -1),
    polyphen_pred=hl.if_else(mt.sortedTranscriptConsequences.size() > 0, mt.sortedTranscriptConsequences[0].polyphen_prediction, hl.missing(hl.tstr)),
    sift_pred=hl.if_else(mt.sortedTranscriptConsequences.size() > 0, mt.sortedTranscriptConsequences[0].sift_prediction, hl.missing(hl.tstr))
)

# annotate rows with additional fields from VEP parent structure before dropping
mt = mt.annotate_rows(
    most_severe_consequence = mt.vep.most_severe_consequence,
    variant_class = mt.vep.variant_class,
    gene_id_array = mt.vep.transcript_consequences.gene_id,
    gene_symbol_array = mt.vep.transcript_consequences.gene_symbol 
)

# drop parent vep structure 
mt = mt.drop(mt.vep)

# VEP filtering is done first as it results in the largest reduction in rows. The call set at the moment is currently
# dominated by variants that are intronic or intergenic.

# Previously the code below had a bunch of string comparison and "or" separating them.
# This is pretty ugly code to do it this way. It's better to have a set and check for membership (using contains).
deleterious_consequences = hl.set(['splice_acceptor_variant', 'splice_donor_variant', 'stop_gained',
                                  'stop_lost','missense_variant', 'frameshift_variant', 'start_lost',
                                  'inframe_insertion', 'inframe_deletion', 'protein_altering_variant'])

#((mt.major_consequence == "missense_variant")&((mt.sift_prediction == "deleterious") | (mt.sift_prediction == "deleterious_low_confidence") | (mt.polyphen_prediction == "probably_damaging") | (mt.polyphen_prediction == "possibly_damaging"))) |
#(mt.major_consequence == "splice_region_variant"), 

# Try to use mt.count() sparingly as it needs to pass over the entire matrix table.
print('Count of the input dataframe before filtering:' + str(mt.count()) + '\n')

# major VEP consequence filtering 
mt = mt.filter_rows(deleterious_consequences.contains(mt.major_consequence),keep = True)
print('Count of the dataframe after also filtering by VEP consequence' + str(mt.count()) + '\n')


# add variant_qc - https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc
mt = hl.variant_qc(mt, name='variant_qc')


# Add in gnomADv2 allele frequencies. This file has annotation for every site in the gemome.

# This hail table is approximately ~300 Gb so passing this file around to the SPARK workers is extremely slow.
# Two ways of speeding this up: 1. subset this hail table down to just generic regions so the hail table will only
# be around 3-10 Gb. 2. Repartition the matrix table we want to annotate. For this project it's currently at
# 30 partitions, thus limiting it to only 30 concurrent jobs that can be run.

combined_ref = hl.read_table('/gpfs/gibbs/pi/brueckner/hail_resources/combined_reference_data_grch38.ht')

mt = mt.annotate_rows(
    gnomad_genomes_af = combined_ref[mt.row_key].gnomad_genomes.AF,
    gnomad_exomes_af = combined_ref[mt.row_key].gnomad_exomes.AF,
    primate_ai = combined_ref[mt.row_key].primate_ai.score,
    splice_ai = combined_ref[mt.row_key].splice_ai.delta_score,
    meta_svm = combined_ref[mt.row_key].dbnsfp.MetaSVM_pred,
    #dbnsfp_polyphen2 = combined_ref[mt.row_key].dbnsfp.Polyphen2_HVAR_pred,
    #dbnsfp_sift = combined_ref[mt.row_key].dbnsfp.SIFT_pred,
    cadd = combined_ref[mt.row_key].cadd.PHRED    
)

mt = mt.annotate_rows(
    gnomad_af = hl.if_else(hl.is_defined(mt.gnomad_exomes_af) | hl.is_defined(mt.gnomad_genomes_af), 
                           hl.if_else(hl.is_defined(mt.gnomad_exomes_af), mt.gnomad_exomes_af, mt.gnomad_genomes_af), 
                           0.0)
    )


# write out filtered matrix table. Note when writing out using Jupyter notebook, need to use the full path otherwise
# it will happily write out without error but the directories are empty!
print('Writes out filtered to the following matrix table')
mt.write('/gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/gruber_WES_b5_vep_conseq_filtered.mt', overwrite = True)


In [ ]:
#Code block that does the non-VEP filtering

mt = hl.read_matrix_table('/gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/gruber_WES_b5_vep_conseq_filtered.mt')

# filter by AC_adj, which excludes variant calls below quality thresholds
# these are: genotype quality (GQ) >= 20, depth (DP) >=10, and allele balance (AB) >= 0.2 and <= 0.8 (for heterozygous genotypes only)
mt = annotate_adj(mt)
mt = mt.annotate_rows(AC_adj = hl.agg.filter(mt.adj, hl.agg.call_stats(mt.GT, mt.alleles).AC[1]))

print('Count of the dataframe BEFORE quality and gnomAD AF filtering:' + str(mt.count()) + '\n')

# filter by PASS status, per https://hail.is/docs/0.2/methods/impex.html
mt = mt.filter_rows((hl.len(mt.filters) == 0) & 
                   (mt.AC_adj > 0) &
                   (mt.gnomad_af <= 0.01), keep=True)

print('Count of the dataframe AFTER quality and gnomAD AF filtering:' + str(mt.count()) + '\n')

# write out filtered matrix table
print('Writes out filtered to the following matrix table')
mt.write('/gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/gruber_WES_b5_vep_filtered_FINAL.mt', overwrite = True)


In [ ]:
# annotate with sample information and write out to file
# read in filtered matrix table
#mt_output = hl_outdir + 'gruber_WES_b5_vep_plus_filtered.mt' 

import hail as hl
mt = hl.read_matrix_table('/gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/gruber_WES_b5_vep_filtered_FINAL.mt')

# sample information 
file_input = 'sample_groups.txt'
print('Reads in the following txt as a hail table: \n' + file_input + '\n')
ht = hl.import_table(file_input).key_by('Adult-ID')
ht = ht.rename({'Adult-ID' : 's'}) #to match mt

# add phenoype group as column annotation
mt = mt.annotate_entries(phenotype = ht[mt.s].group)
mt.describe()

# annotate with AC_adj for each phenotype group
mt = mt.annotate_rows(AC_adj_Barlow = hl.agg.filter((mt.phenotype == "Barlow") & mt.adj, hl.agg.call_stats(mt.GT, mt.alleles).AC[1]))
mt = mt.annotate_rows(AC_adj_FED = hl.agg.filter((mt.phenotype == "FED") & mt.adj, hl.agg.call_stats(mt.GT, mt.alleles).AC[1]))
mt.rows().show(5)

# annotate if in gene list
ht = hl.import_table('gruber_gene_list.txt',delimiter='\t').key_by('Gene_Symbol')
ht = ht.rename({'Gene_Symbol' : 'gene_symbol'}) #to match mt
ht.show()
mt = mt.annotate_rows(
    in_gene_list = hl.if_else(hl.is_defined(ht[mt.gene_symbol].gene_annotation), 'YES', 'NO'),
    gene_annotation = hl.if_else(hl.is_defined(ht[mt.gene_symbol].gene_annotation), ht[mt.gene_symbol].gene_annotation, 'NA')
)
mt.describe()


# Frequently mutated genes. https://bmcmedgenomics.biomedcentral.com/articles/10.1186/s12920-014-0064-y
ht = hl.import_table('flag_gene_list.txt',delimiter='\t').key_by('gene_symbol')
ht.show()
mt = mt.annotate_rows(
    flag_gene_list = hl.if_else(hl.is_defined(ht[mt.gene_symbol]), 'YES', 'NO'),
)
mt.describe()


# use make_table to make a table from a matrix table with one field per sample.
# perhaps this could be used to get variants calls per sample, subset for those with GT 0/1 or 1/1
#mt = mt.make_table()

# save as tsv
#final_output = hl_outdir + 'mvd_WES_analysis.tsv' 
table = mt.rows()
table.show()
table = table.drop('filters','allele_data','a_index','was_split','vep_proc_id','sortedTranscriptConsequences','gene_id_array','gene_symbol_array')
table.show()
#table.write('/gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/mvd_WES_analysis.ht')
table.export('/gpfs/ycga/scratch60/ysm/lek/ml2529/gruber_analysis/mvd_WES_analysis_v3.tsv')